# Fletcher Project: Tweets Summarizing

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.corpus import treebank_chunk
from textblob import TextBlob
import pymongo
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from operator import itemgetter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from __future__ import division, unicode_literals
import operator
import requests
from requests_oauthlib import OAuth1
from pymongo import MongoClient
import tweepy
import cnfg
import numpy as np
import re

In [499]:
client = MongoClient()

In [500]:
datatweets = client.dsbc.datatweets

In [501]:
datatweets.remove( { } )

{u'n': 50, u'ok': 1}

In [502]:
datatweets = client.dsbc.datatweets
for k in tweets:
    datatweets_d = {'id': k}
    datatweets.insert(datatweets_d)

In [503]:
datatweets

Collection(Database(MongoClient('localhost', 27017), u'dsbc'), u'datatweets')

In [504]:
datatweets.count()

200

## Getting Twitter Data

In [455]:
config = cnfg.load(".twitter_config")

In [456]:
auth = tweepy.OAuthHandler(config['consumer_key'],
              config['consumer_secret'])
auth.set_access_token(config['access_token'],
              config['access_token_secret'])

In [457]:
api = tweepy.API(auth)

In [458]:
account = 'TheEconomist'
data_tweets = api.user_timeline(account, count=200)

## Setting chosen keyword

In [478]:
keyword ='china'

In [479]:
#data_tweets

In [480]:
tweets = []
tweets_text = []
for tweet in data_tweets:
    tweets.append(tweet.text)
#    tweets.append([tweet.text, tweet.retweet_count, tweet.favorite_count])
tweets

[u"Obama's immigration order is now tied up in courts; this may hurt GOP candidates for 2016 http://t.co/rBxLVX0B6B http://t.co/6JTJ7os9SR",
 u"It is a sweet moment for Europe's Green parties. They shouldn't throw caution to the winds http://t.co/3StZ05O3PU http://t.co/q30srRzvLE",
 u'Veer Savarkar: The man who thought Gandhi a sissy http://t.co/4H1tHIwhVP http://t.co/VyRI7p8GAS',
 u'Never mind planets. The solar system\u2019s icy moons may be the best places to hunt for aliens http://t.co/ZJ3d9n364m http://t.co/Fyz8MmKOa9',
 u'Sponsors consider their association with FIFA, today\u2019s #Dailychart looks at its balance sheet http://t.co/1xy8hqgrq0 http://t.co/24ZkZZWOTm',
 u'Many cities are losing inhabitants. Better to manage decline than try to stop it http://t.co/a0wR6rAtXo http://t.co/iugbnUtsLr',
 u'The Economist explains why India\u2019s heatwaves are so deadly http://t.co/g4FnBUSSCK http://t.co/0EyYeRpbm8',
 u'The average new mother has become substantially older http://t.co/NzQ

In [481]:
len(tweets)

200

In [482]:
stopw = [str('http'), str('https'), str('['), str(']'), str('rt'), str('%'), str('@'), str('#')]
stopngram = stopwords.words('english')
stopngram += ['.', ',', '(', ')', "'", '"', 'http', 'RT']

In [483]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and "\\" not in r"%r" % keys:
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1

In [484]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:10]:
    print '%20s %i' % (words, count)

                week 10
               china 9
           economist 9
             america 9
               india 8
             britain 8
           companies 7
               death 7
              people 7
         stockmarket 7


In [485]:
#tweets_t = zip(*tweets)[0]
#tweets_r = zip(*tweets)[1]

In [486]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 2
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

       The Economist 9
  Economist explains 8
            China 's 8
             week 's 7
    alliances rivals 6
  Cheeseburger bills 6
       death penalty 6
          Britain 's 6
        The pressure 5
irrational exuberance 5


In [487]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 3
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

The Economist explains 8
form alliances rivals 5
    China 's booming 5
's booming stockmarket 5
pressure companies form 5
companies form alliances 5
alliances rivals growing 5
rivals growing inexorably 5
The pressure companies 5
  looks like classic 4


In [488]:
keyword = str(keyword)
keyword = keyword.lower()
keyword

'china'

In [489]:
tweets_match = []
for r in tweets:
    if keyword in r.lower():
        tweets_match.append(r)

len(tweets_match)

12

In [490]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets_match:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and keyword not in keys and "\\" not in r"%r" % keys:
            if tags == 'JJ':
                adjectives.setdefault(keys,0)
                adjectives[keys] += 1
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1
            if tags == 'RB':
                adverbs.setdefault(keys,0)
                adverbs[keys] += 1
            if tags == 'VB':
                verbs.setdefault(keys,0)
                verbs[keys] += 1

In [491]:
for words, count in sorted(verbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

             booming 5
                  is 4
                says 2
                fall 2
           exploring 1


In [492]:
for words, count in sorted(adjectives.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

          irrational 4
             classic 4
                much 2
                more 2
                real 1


In [493]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

         stockmarket 6
               looks 4
          exuberance 4
                case 4
                 law 2


In [494]:
for words, count in sorted(adverbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                 ago 1
                 now 1


In [495]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 2
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

            China 's 8
 booming stockmarket 5
          's booming 5
        classic case 4
irrational exuberance 4


In [496]:
stop = stopwords.words('english')

counter = defaultdict(int)

n = 3
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

    China 's booming 5
's booming stockmarket 5
   like classic case 4
  looks like classic 4
        How China 's 4
